# Portland Asthma Study

### There are multiple risk factors for asthma including social, environmental and financial. [1] Portland Oregon is an urban area which takes pride in protecting people from environment exposure to toxic chemicals from fuel exhaust among other sources. It has 92,000 acres of green spaces and "...the highest rate of biking to work of any major US city."[2] This study will look at whether these efforts have had an impact in reducing rates of adult asthma.

### Sources
1. Malik HU, Kumar K, Frieri M. Minimal difference in the prevalence of asthma in the urban and rural environment. Clin Med Insights Pediatr. 2012 Jun 19;6:33-9. doi: 10.4137/CMPed.S9539. PMID: 23641164; PMCID: PMC3620776.
2. Green City Times, available online at (https://www.greencitytimes.com/portland/)

In [1]:
# Import packages for modeling and statistical analysis of spatial data
import os
import pathlib
import time
import warnings

import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pystac_client
import requests
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import scipy.stats as stats
import shapely
import statsmodels.api as sm
import xarray as xr
from cartopy import crs as ccrs
from holoviews import opts
from holoviews.element import tiles
from scipy.ndimage import convolve
from sklearn.model_selection import KFold
from scipy.ndimage import label
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

data_dir = os.path.join(
    pathlib.Path.home(),
    'Documents',
    'eaclassprojects',
    'bigdata',
    'portland'
    )
os.makedirs(data_dir, exist_ok=True)
    
warnings.simplefilter('ignore')

# Prevent GDAL from quitting due to momentary disruptions
os.environ["GDAL_HTTP_MAX_RETRY"] = "5"
os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"

c:\Users\stem2\miniconda3\envs\earth-analytics-python\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
# Set up the census tract path
tract_dir = os.path.join(data_dir, 'portland-tract')
os.makedirs(tract_dir, exist_ok=True)
tract_path = os.path.join(tract_dir, 'portland-tract.shp')

# Download the census tracts (only once)
if not os.path.exists(tract_path):
    tract_url = ('https://data.cdc.gov/download/x7zy-2xmx/application%2Fzip')
    tract_gdf = gpd.read_file(tract_url)
    po_tract_gdf = tract_gdf[tract_gdf.PlaceName=='Portland']
    po_tract_gdf.to_file(tract_path, index=False)

# Load in the census tract data
po_tract_gdf = gpd.read_file(tract_path)

# Site plot -- Census tracts with satellite imagery in the background
(
    po_tract_gdf
    .to_crs(ccrs.Mercator())
    .hvplot(
        line_color='orange', fill_color=None, 
        crs=ccrs.Mercator(), tiles='EsriImagery',
        frame_width=600)
)

po_tract_gdf.head()

,place2010,tract2010,ST,PlaceName,plctract10,PlcTrPop10,geometry
0,2360545,23005000100,23,Portland,2360545-23005000100,2346,"MULTIPOLYGON (((-7819264.773 5414638.107, -781..."
1,2360545,23005000200,23,Portland,2360545-23005000200,2349,"POLYGON ((-7819262.929 5413477.756, -7819269.5..."
2,2360545,23005000300,23,Portland,2360545-23005000300,2625,"POLYGON ((-7819262.929 5413477.756, -7819348.6..."
3,2360545,23005000500,23,Portland,2360545-23005000500,2420,"MULTIPOLYGON (((-7820627.306 5414284.438, -782..."
4,2360545,23005000600,23,Portland,2360545-23005000600,3389,"POLYGON ((-7820810.983 5413214.16, -7820836.03..."


In [3]:
# Set up a path for asthma data
cdc_path = os.path.join(data_dir, 'asthma.csv')

# Download asthma data (only once)
if not os.path.exists(cdc_path):
    cdc_url = (
        "https://data.cdc.gov/resource/cwsq-ngmh.csv"
        "?year=2022"
        "&stateabbr=OR"
        "&countyname=Multnomah"
        "&measureid=CASTHMA"
        "&$limit=1500"
    )
    cdc_df = (
        pd.read_csv(cdc_url)
        .rename(columns={
            'data_value': 'asthma',
            'low_confidence_limit': 'asthma_ci_low',
            'high_confidence_limit': 'asthma_ci_high',
            'locationname': 'tract'})
        [[
            'year', 'tract', 
            'asthma', 'asthma_ci_low', 'asthma_ci_high', 'data_value_unit',
            'totalpopulation', 'totalpop18plus'
        ]]
    )
    cdc_df.to_csv(cdc_path, index=False)

# Load asthma data
cdc_df = pd.read_csv(cdc_path)

# Preview asthma data
cdc_df

,year,tract,asthma,asthma_ci_low,asthma_ci_high,data_value_unit,totalpopulation,totalpop18plus
0,2022,41051000202,11.4,10.1,12.8,%,3456,2875
1,2022,41051000702,10.8,9.6,12.1,%,5243,4268
2,2022,41051000602,11.5,10.2,12.9,%,5756,4612
3,2022,41051000402,10.3,9.1,11.5,%,3906,3188
4,2022,41051000401,10.4,9.2,11.6,%,3746,3023
...,...,...,...,...,...,...,...,...
192,2022,41051008904,11.2,10.0,12.6,%,5006,3829
193,2022,41051001602,10.7,9.5,12.0,%,4616,3786
194,2022,41051001302,10.5,9.4,11.8,%,3376,2823
195,2022,41051001202,10.3,9.1,11.5,%,3342,2804


In [4]:
# Query CDC api
def check_api_functionality(url, expected_status_code=200):  # Default to 200 OK
    try:
        response = requests.get(url)  # Or the appropriate method for your API
        response.raise_for_status() # Check for HTTP errors

        if response.status_code == expected_status_code:
            print(f"API endpoint {url} is working correctly (returned {response.status_code}).")
            # You can now inspect the response content (JSON, XML, etc.)
            try:
                data = response.json()  # Try to parse JSON (if it's a JSON API)
                # Further process and validate the 'data' as needed
                print("API returned JSON data (sample):", data[:2]) # Print a small sample
                return True
            except ValueError:
                print("API returned non-JSON data.")
                return True # Consider this a success, but handle non-JSON appropriately
        else:
            print(f"API endpoint {url} returned unexpected status code: {response.status_code}")
            return False

    except requests.exceptions.RequestException as e:
        print(f"Error checking API URL {url}: {e}")
        return False 
        
api_url = "https://data.cdc.gov/resource/cwsq-ngmh.csv"   
if check_api_functionality(api_url): # Uses default 200 code check
    print("API functional check passed.")
else:
    print("API functional check failed.")


api_url = "https://data.cdc.gov/resource/cwsq-ngmh.csv" # Example URL that returns a 404 error.
if check_api_functionality(api_url): # Uses default 200 code check
    print("API functional check passed.")
else:
    print("API functional check failed.")

API endpoint https://data.cdc.gov/resource/cwsq-ngmh.csv is working correctly (returned 200).
API returned non-JSON data.
API functional check passed.
API endpoint https://data.cdc.gov/resource/cwsq-ngmh.csv is working correctly (returned 200).
API returned non-JSON data.
API functional check passed.


In [5]:
# Change tract identifier datatype for merging
po_tract_gdf.tract2010 = po_tract_gdf.tract2010.astype('int64')

# Merge census data with geometry
tract_cdc_gdf = (
    po_tract_gdf
    .merge(cdc_df, left_on='tract2010', right_on='tract', how='inner')
)

# Plot asthma data as chloropleth
(
    gv.tile_sources.EsriImagery
    * 
    gv.Polygons(
        tract_cdc_gdf.to_crs(ccrs.Mercator()),
        vdims=['asthma', 'tract2010'],
        crs=ccrs.Mercator()
    ).opts(color='asthma', colorbar=True, tools=['hover'])
).opts(title='Asthma Rates', width=600, height=600)



:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (asthma,tract2010)

In [6]:
# Connect to the planetary computer catalog
e84_catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
e84_catalog.title

'Microsoft Planetary Computer STAC API'

In [7]:
# Convert geometry to lat/lon for STAC
tract_area_gdf = tract_cdc_gdf.to_crs(4326)

# Define a path to save NDVI stats
ndvi_stats_path = os.path.join(data_dir, 'portland-ndvi-stats.csv')

# Check for existing data - do not access duplicate tracts
downloaded_tracts = []
if os.path.exists(ndvi_stats_path):
    ndvi_stats_df = pd.read_csv(ndvi_stats_path)
    downloaded_tracts = ndvi_stats_df.tract.values
else:
    print('No census tracts downloaded so far')

tract_cdc_gdf.head()

No census tracts downloaded so far


,place2010,tract2010,ST,PlaceName,plctract10,PlcTrPop10,geometry,year,tract,asthma,asthma_ci_low,asthma_ci_high,data_value_unit,totalpopulation,totalpop18plus
0,4159000,41051000301,41,Portland,4159000-41051000301,5041,"POLYGON ((-13650305.147 5699042.371, -13650193...",2022,41051000301,11.2,10.0,12.6,%,5443,4908
1,4159000,41051000302,41,Portland,4159000-41051000302,6709,"POLYGON ((-13650063.25 5697279.135, -13649943....",2022,41051000302,10.1,9.0,11.3,%,7191,5476
2,4159000,41051000401,41,Portland,4159000-41051000401,3418,"POLYGON ((-13648772.389 5699042.688, -13648767...",2022,41051000401,10.4,9.2,11.6,%,3746,3023
3,4159000,41051000402,41,Portland,4159000-41051000402,3477,"POLYGON ((-13647657.747 5699037.289, -13647659...",2022,41051000402,10.3,9.1,11.5,%,3906,3188
4,4159000,41051000501,41,Portland,4159000-41051000501,3732,"POLYGON ((-13646585.183 5696754.598, -13646694...",2022,41051000501,11.1,9.9,12.4,%,4134,3307


In [10]:
# Convert geometry to lat/lon for STAC
tract_latlon_gdf = tract_cdc_gdf.to_crs(4326)

# Define a path to save NDVI stats
ndvi_stats_path = os.path.join(data_dir, 'asthma.csv')

# Check for existing data - do not access duplicate tracts
downloaded_tracts = []
if os.path.exists(ndvi_stats_path):
    ndvi_stats_df = pd.read_csv(ndvi_stats_path)
    downloaded_tracts = ndvi_stats_df.tract.values
else:
    print('No census tracts downloaded so far')
    
# Loop through each census tract
scene_dfs = []
for i, tract_values in tqdm(tract_latlon_gdf.iterrows()):
    tract = tract_values.tract2010
    # Check if statistics are already downloaded for this tract
    if not (tract in downloaded_tracts):
        # Retry up to 5 times in case of a momentary disruption
        i = 0
        retry_limit = 5
        while i < retry_limit:
            # Try accessing the STAC
            try:
                # Search for tiles
                naip_search = e84_catalog.search(
                    collections=["naip"],
                    intersects=shapely.to_geojson(tract_values.geometry),
                    datetime="2021"
                )
                
                # Build dataframe with tracts and tile urls
                scene_dfs.append(pd.DataFrame(dict(
                    tract=tract,
                    date=[pd.to_datetime(scene.datetime).date() 
                          for scene in naip_search.items()],
                    rgbir_href=[scene.assets['image'].href for scene in naip_search.items()],
                )))
                
                break
            # Try again in case of an APIError
            except pystac_client.exceptions.APIError:
                print(
                    f'Could not connect with STAC server. '
                    f'Retrying tract {tract}...')
                time.sleep(2)
                i += 1
                continue
    
# Concatenate the url dataframes
if scene_dfs:
    scene_df = pd.concat(scene_dfs).reset_index(drop=True)
else:
    scene_df = None

# Preview the URL DataFrame
scene_df

0it [00:00, ?it/s]

In [11]:
# Skip this step if data are already downloaded 
if not scene_df is None:
    ndvi_dfs = []
    # Loop through the census tracts with URLs
    for tract, tract_date_gdf in tqdm(scene_df.groupby('tract')):
        # Open all images for tract
        tile_das = []
        for _, href_s in tract_date_gdf.iterrows():
            # Open vsi connection to data
            tile_da = rxr.open_rasterio(
                href_s.rgbir_href, masked=True).squeeze()
            
            # Clip data
            boundary = (
                tract_cdc_gdf
                .set_index('tract2010')
                .loc[[tract]]
                .to_crs(tile_da.rio.crs)
                .geometry
            )
            crop_da = tile_da.rio.clip_box(
                *boundary.envelope.total_bounds,
                auto_expand=True)
            clip_da = crop_da.rio.clip(boundary, all_touched=True)
                
            # Compute NDVI
            ndvi_da = (
                (clip_da.sel(band=4) - clip_da.sel(band=1)) 
                / (clip_da.sel(band=4) + clip_da.sel(band=1))
            )

            # Accumulate result
            tile_das.append(ndvi_da)

        # Merge data
        scene_da = rxrmerge.merge_arrays(tile_das)

        # Mask vegetation
        veg_mask = (scene_da>.3)

        # Calculate statistics and save data to file
        total_pixels = scene_da.notnull().sum()
        veg_pixels = veg_mask.sum()

        # Calculate mean patch size
        labeled_patches, num_patches = label(veg_mask)
        # Count patch pixels, ignoring background at patch 0
        patch_sizes = np.bincount(labeled_patches.ravel())[1:] 
        mean_patch_size = patch_sizes.mean()

        # Calculate edge density
        kernel = np.array([
            [1, 1, 1], 
            [1, -8, 1], 
            [1, 1, 1]])
        edges = convolve(veg_mask, kernel, mode='constant')
        edge_density = np.sum(edges != 0) / veg_mask.size
        
        # Add a row to the statistics file for this tract
        pd.DataFrame(dict(
            tract=[tract],
            total_pixels=[int(total_pixels)],
            frac_veg=[float(veg_pixels/total_pixels)],
            mean_patch_size=[mean_patch_size],
            edge_density=[edge_density]
        )).to_csv(
            ndvi_stats_path, 
            mode='a', 
            index=False, 
            header=(not os.path.exists(ndvi_stats_path))
        )

# Re-load results from file
ndvi_stats_df = pd.read_csv(ndvi_stats_path)
ndvi_stats_df

,year,tract,asthma,asthma_ci_low,asthma_ci_high,data_value_unit,totalpopulation,totalpop18plus
0,2022,41051000202,11.4,10.1,12.8,%,3456,2875
1,2022,41051000702,10.8,9.6,12.1,%,5243,4268
2,2022,41051000602,11.5,10.2,12.9,%,5756,4612
3,2022,41051000402,10.3,9.1,11.5,%,3906,3188
4,2022,41051000401,10.4,9.2,11.6,%,3746,3023
...,...,...,...,...,...,...,...,...
192,2022,41051008904,11.2,10.0,12.6,%,5006,3829
193,2022,41051001602,10.7,9.5,12.0,%,4616,3786
194,2022,41051001302,10.5,9.4,11.8,%,3376,2823
195,2022,41051001202,10.3,9.1,11.5,%,3342,2804


In [13]:
# Merge census data with geometry
ndvi_cdc_gdf = (
    tract_cdc_gdf
    .merge(
        ndvi_stats_df,
        left_on='tract2010', right_on='tract', how='inner')
)

# Plot chloropleths with vegetation statistics
def plot_chloropleth(gdf, **opts):
    """Generate a chloropleth with the given color column"""
    return gv.Polygons(
        gdf.to_crs(ccrs.Mercator()),
        crs=ccrs.Mercator()
    ).opts(colorbar=True, **opts)

(
    plot_chloropleth(
        ndvi_cdc_gdf, title='Asthma Rates', color='asthma', cmap='viridis')
    + 
    plot_chloropleth(ndvi_cdc_gdf, title='Vegetation Index', color='edge_density', cmap='Greens')
)

:Layout
   .Polygons.I  :Polygons   [Longitude,Latitude]   (place2010,tract2010,ST,PlaceName,plctract10,PlcTrPop10,year_x,tract_x,asthma_x,asthma_ci_low_x,asthma_ci_high_x,data_value_unit_x,totalpopulation_x,totalpop18plus_x,year_y,tract_y,asthma_y,asthma_ci_low_y,asthma_ci_high_y,data_value_unit_y,totalpopulation_y,totalpop18plus_y)
   .Polygons.II :Polygons   [Longitude,Latitude]   (place2010,tract2010,ST,PlaceName,plctract10,PlcTrPop10,year_x,tract_x,asthma_x,asthma_ci_low_x,asthma_ci_high_x,data_value_unit_x,totalpopulation_x,totalpop18plus_x,year_y,tract_y,asthma_y,asthma_ci_low_y,asthma_ci_high_y,data_value_unit_y,totalpopulation_y,totalpop18plus_y)